# Movielens 영화 추천🎬

## 루브릭 기준

1. CSR matrix가 정상적으로 만들어졌다.
사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.

2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.
사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.

3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.
MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.

***

In [1]:
import os
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

## 1. 데이터 준비와 전처리

사용할 데이터셋은 Movielens 데이터이다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 존재한다
- 별점 데이터는 대표적인 explicit 데이터이나, implicit 데이터로 간주하고 테스트한다.
- 별점을 시청횟수로 해석해서 생각한다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외한다.

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# ratings 와 movies 데이터 프레임 합치기
data = ratings.join(movies.set_index('movie_id'), on='movie_id')
data.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [7]:
# user_id가 1인 사용자가 어떤 영화를 시청했는지 확인해봅니다.
movie_id_user1 = data[data['user_id']== 1]['title']
movie_id_user1

0                One Flew Over the Cuckoo's Nest (1975)
1                      James and the Giant Peach (1996)
2                                   My Fair Lady (1964)
3                                Erin Brockovich (2000)
4                                  Bug's Life, A (1998)
5                            Princess Bride, The (1987)
6                                        Ben-Hur (1959)
7                             Christmas Story, A (1983)
8                Snow White and the Seven Dwarfs (1937)
9                              Wizard of Oz, The (1939)
10                          Beauty and the Beast (1991)
11                                          Gigi (1958)
12                        Miracle on 34th Street (1947)
13                      Ferris Bueller's Day Off (1986)
14                           Sound of Music, The (1965)
15                                     Airplane! (1980)
16                                        Tarzan (1999)
17                                         Bambi

## 2. 데이터 탐색하기

- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [8]:
# 영화 개수
print('영화 개수: ', data['movie_id'].nunique())

영화 개수:  3628


In [9]:
# 사용자 수
print('사용자 수: ', data['user_id'].nunique())

사용자 수:  6039


In [10]:
# 인기있는 영화 Top 30
print('인기있는 영화 Top 30')
movie_count = data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

인기있는 영화 Top 30


title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [11]:
# 유저별 몇 편의 영화를 시청했는지에 대한 통계
user_count = data.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [12]:
# 영화 별 시청(count)횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['count'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: count, dtype: float64

## 3. 모델 검증을 위한 사용자 초기 정보 세팅
내가 선호하는 영화를 5가지 골라서 rating에 추가해준다. 예측값 해석을 용이하게 하기 위해 애니메이션 장르의 영화 5개를 초기 정보값으로 입력해 주었다.

In [71]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('lion', regex=False)]

,movie_id,title,genre
360,364,"Lion King, The (1994)",Animation|Children's|Musical
1980,2049,"Happiest Millionaire, The (1967)",Comedy|Musical


In [14]:
# 좋아하는 영화의 id 리스트
my_favoite_id = [1, 2294, 1566, 588, 1907]

# id 리스트를 title 리스트로 변환해준다
my_favorite_movie = []
for mid in my_favoite_id:
    my_favorite_movie.append(list(movies[movies['movie_id']==mid]['title'])[0])
    
# '6041'6이라는 user_id가 위 영화를 5회씩 시청했다고 가정하겠습니다.
my_watchlist = pd.DataFrame({'user_id': ['6041']*5, 'movie_id': my_favoite_id, 'title': my_favorite_movie, 'count':[5]*5})

if not data.isin({'user_id':['6041']})['user_id'].any():  # user_id에 'eunyoeng'이라는 데이터가 없다면
    data = data.append(my_watchlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data = data[['user_id', 'movie_id', 'title', 'count']]
data.tail()

,user_id,movie_id,title,count
0,6041,1,Toy Story (1995),5
1,6041,2294,Antz (1998),5
2,6041,1566,Hercules (1997),5
3,6041,588,Aladdin (1992),5
4,6041,1907,Mulan (1998),5


## 4. CSR(Compressed Sparse Row) Matrix 만들기

![csr matrix](https://lovit.github.io/assets/figures/sparse_matrix_csr.png)
출처:https://lovit.github.io/nlp/machine%20learning/2018/04/09/sparse_mtarix_handling/#csr-matrix

CSR Matrix는 sparse한 matrix에서 유효한 데이터의 값과 과표 정보만으로 구성되어 메모리 사용량을 최소화할 수 있는 데이터 구조이다. row 순서대로 데이터를 저장한다.

In [15]:
# 고유한 유저, 영화를 찾기
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

In [45]:
# 유저, 영화 indexing
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [17]:
# 데이터 컬럼 내 값을 indexing된 값으로 교체

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['new_user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')
    
# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('title column indexing OK!!')
    data['new_movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,title,count,new_user_id,new_movie_id
0,1,1193,One Flew Over the Cuckoo's Nest (1975),5,0,0
1,1,661,James and the Giant Peach (1996),3,0,1
2,1,914,My Fair Lady (1964),3,0,2
3,1,3408,Erin Brockovich (2000),4,0,3
4,1,2355,"Bug's Life, A (1998)",5,0,4
...,...,...,...,...,...,...
0,6041,1,Toy Story (1995),5,6039,40
1,6041,2294,Antz (1998),5,6039,30
2,6041,1566,Hercules (1997),5,6039,32
3,6041,588,Aladdin (1992),5,6039,33


In [18]:
# CSR Matrix 생성
num_user = data['new_user_id'].nunique()
num_movie = data['new_movie_id'].nunique()

# csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
csr_data = csr_matrix((data['count'], (data.new_user_id, data.new_movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. MF 모델 학습하기

![Fast and Scalable Matrix Factorization](https://d3s0tskafalll9.cloudfront.net/media/images/E-3v2-2_ekCv9hW.max-800x600.png)
출처:https://arxiv.org/pdf/1610.05838.pdf

- (m,n) 사이즈의 행렬 R을 (m,k) 사이즈의 행렬 P와 (k,n) 사이즈의 행렬 Q로 분해한다면 R은 P와 Q의 행렬곱으로 표현 가능할 수 있다.
- 이때의 P와 Q는 Feature Matrix
- 영화를 추천하는 모델에 대입한다면, 사용자의 특성(Feature) 벡터인 P, 영화의 특성 벡터인 Q로 볼 수 있고 두 벡터를 내적해서 얻어지는 R이 사용자의 영화 선호도를 보는 것
- k는 m,n보다 작은 값이기 때문에 계산량 측면으로도 훨씬 유리하다.
- MF 모델은 성능이 준수하고 Scalability가 좋아서 많이 사용된다.

- 모델의 **목표**는 모든 유저와 아이템에 대해  **k-dimension의 벡터를 잘 만드는 것**
- 벡터를 잘 만든다는 기준은 유저i의 벡터$U_i$와 아이템j의 벡터$I_j$를 내적했을 때 유저i가 아이템j에 대해 평가한 수치$M_ij$와 비슷한가이다.
$$U_i*I_j=M_{ij}$$

***

Matrix Factorization 모델을 implicit 패키지를 사용하여 학습해 본다.
- `implicit` 패키지는 implicit dataset을 사용하는 다양한 모델을 빠르게 학습할 수 있는 패키지
- `als(AlternatingLeastSquares) model`: MF에서 두 Feature Matrix를 한번에 훈련하는 것은 잘 수렴하지 않아, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 방식(AlternatingLeastSquares)이 효과적이다.
>__Implicit Datasets__<br>
Implicit Data는 유저가 간접적(Implicit)으로 선호, 취향을 나타내는 데이터를 의미한다. 예시로는 검색 기록, 방문 페이지, 구매 내역 등이 있다
    
    출처: https://orill.tistory.com/entry/Explicit-vs-Implicit-Feedback-Datasets?category=1066301

In [19]:
# implicit 라이브러리에서 권장하고 있는 세팅
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

__AlternatingLeastSquares__ 클래스의 **\_\_init\_\_ 파라미터**를 살펴보면,

1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
3. use_gpu : GPU를 사용할 것인지 
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

1,4를 늘릴수록 학습을 잘 하게 되지만 과적합의 우려가 있다.

In [20]:
AlternatingLeastSquares.__init__

<function implicit.als.AlternatingLeastSquares.__init__(self, factors=100, regularization=0.01, dtype=<class 'numpy.float32'>, use_native=True, use_cg=True, use_gpu=False, iterations=15, calculate_training_loss=False, num_threads=0, random_state=None)>

### 모델 설계

In [31]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

In [32]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

### 모델 학습

In [33]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

## 6. 결과 확인

### 6-1. 영화 선호도 파악하기

In [34]:
eunyeong, mulan = user_to_idx['6041'], movie_to_idx['Mulan (1998)']
eunyeong_vector, mulan_vector = als_model.user_factors[eunyeong], als_model.item_factors[mulan]

In [35]:
eunyeong_vector # 나의 사용자 벡터

array([-3.90605122e-01, -4.31402549e-02, -5.47213778e-02,  2.77361959e-01,
        1.79703869e-02, -2.50509828e-02,  3.45937639e-01, -3.66632044e-01,
       -9.32605863e-02, -6.67776689e-02, -3.63928109e-01, -4.24080491e-01,
        2.81437576e-01,  9.36534226e-01, -3.91867757e-02,  2.61897258e-02,
       -5.71133271e-02,  5.23032129e-01, -4.90260184e-01, -5.75039744e-01,
       -8.33045989e-02, -5.67516923e-01,  1.23528406e-01, -6.72973767e-02,
       -1.94550641e-02, -7.14734674e-01,  3.19132179e-01, -4.89674360e-01,
        5.71150362e-01,  1.64225116e-01, -6.66151464e-01, -6.87472701e-01,
        3.40279132e-01,  4.93038505e-01,  6.94664836e-01,  7.70274818e-01,
        1.78950578e-01, -1.70772821e-01,  1.69495076e-01,  7.22523749e-01,
        2.80270785e-01,  4.14711207e-01,  2.89038360e-01,  4.17430580e-01,
       -7.42264271e-01,  2.61259317e-01,  1.80783883e-01,  6.82639003e-01,
        4.43345994e-01,  7.66689554e-02, -2.62236893e-01,  6.54647127e-02,
        8.28569159e-02, -

In [36]:
mulan_vector # 영화 뮬란의 벡터

array([ 6.96733885e-04,  9.93920024e-03,  8.21210488e-05,  2.07405128e-02,
        4.04278701e-03,  8.45363177e-03,  6.75080018e-03,  2.18260637e-03,
        7.49140978e-03,  1.55579648e-03, -6.49726717e-03, -2.20742784e-02,
        1.97737366e-02,  2.78198738e-02, -4.61074151e-03,  1.55536588e-02,
       -2.99386773e-03,  6.64541358e-03, -1.67546654e-03,  1.08125536e-02,
       -1.10680843e-02, -2.04611402e-02, -8.94658267e-03,  5.30994264e-03,
       -5.87085821e-03, -1.10131735e-02,  5.51885692e-03, -1.04553038e-02,
        1.85099915e-02,  2.29236926e-03, -1.56375188e-02, -7.50134187e-03,
        8.10484122e-03,  6.29697274e-03,  1.83211826e-02,  4.24057432e-02,
        1.11224344e-02,  9.61569045e-03,  7.28878519e-03,  8.23131762e-03,
        1.32358689e-02, -4.03971737e-03,  1.28876017e-02,  1.04650147e-02,
       -1.04570771e-02,  6.57843612e-03,  1.66645285e-03,  2.03032177e-02,
        9.99143068e-03,  1.10408226e-02,  5.82855567e-03, -1.57057552e-03,
        4.33494523e-03,  

In [37]:
# 나와 뮬란의 내적 구하기 (선호 리스트에 있는 영화)
np.dot(eunyeong_vector, mulan_vector)

0.6135275

In [38]:
# 모델이 나의 titanic 대한 선호도를 어떻게 예측하는지 확인(선호 리스트에 없는 영화)
titanic = movie_to_idx['Titanic (1997)']
titanic_vector = als_model.item_factors[titanic]
np.dot(eunyeong_vector, titanic_vector)

0.11037781

In [39]:
# 모델이 나의 Bug's Life대한 선호도를 어떻게 예측하는지 확인
bugs_life = movie_to_idx["Bug's Life, A (1998)"]
bugs_life_vector = als_model.item_factors[bugs_life]
np.dot(eunyeong_vector, bugs_life_vector)

0.4887701

In [40]:
# 모델이 나의 Matrix 대한 선호도를 어떻게 예측하는지 확인
matrix = movie_to_idx['Matrix, The (1999)']
matrix_vector = als_model.item_factors[matrix]
np.dot(eunyeong_vector, matrix_vector)

-0.010847442

선호하는 영화 리스트에 있는 Mulan에 대한 선호도는 0.6정도이다.
Bugs' Life에 대한 예측된 선호도는 0.49정도로
Matrix 나 Titanic에 대한 예측값이 0.11, -0.01인것을 보면 높은 선호도를 보일 것이라 예측 해준다. 
사용자 초기 정보 세팅할 때 선호하는 영화 5개를 모두 장르가 `Animation|Children's`에 속하는 영화로  넣어주었기 때문에, 나는 애니메이션을 매우 좋아하는 사용자라고 학습되었고, 그 결과 애니메이션 장르인 Bug's Life는 높은 선호도를 가질 것이라 예측 해준것 같다.
`Drama|Romance`인 Titanic 보다 `Action|Sci-Fi|Thriller`인 Matrix의 선호도가 낮을 거라 예측해준 것도 흥미롭다.

### 6-2. 내가 좋아하는 영화와 비슷한 영화 추천

`AlternatingLeastSquares` 클래스에 구현되어 있는 `similar_items` 메서드를 통하여 비슷한 영화를 찾는다.

In [46]:
def get_similar_artist(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    idx_to_movie = {v:k for k,v in movie_to_idx.items()}
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [47]:
get_similar_artist('Mulan (1998)')

['Mulan (1998)',
 'Tarzan (1999)',
 'Hunchback of Notre Dame, The (1996)',
 'Hercules (1997)',
 'Rescuers Down Under, The (1990)',
 'Antz (1998)',
 'Anastasia (1997)',
 'Beauty and the Beast (1991)',
 'Lion King, The (1994)',
 'Aladdin and the King of Thieves (1996)']

In [48]:
get_similar_artist('Titanic (1997)')

['Titanic (1997)',
 'Ever After: A Cinderella Story (1998)',
 'Snow Day (2000)',
 'Jerry Maguire (1996)',
 'City of Angels (1998)',
 'English Patient, The (1996)',
 'Walking Dead, The (1995)',
 'Idolmaker, The (1980)',
 "You've Got Mail (1998)",
 "Mr. Holland's Opus (1995)"]

In [49]:
get_similar_artist('Matrix, The (1999)')

['Matrix, The (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Fugitive, The (1993)',
 'Fifth Element, The (1997)',
 'Terminator, The (1984)',
 'Face/Off (1997)',
 'Twelve Monkeys (1995)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Men in Black (1997)']

### 6-3. 내가 좋아할 만한 영화는?

In [55]:
user = user_to_idx['6041']

# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)

# 인덱스를 영화 제목으로 변환
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in movie_recommended]

['Lion King, The (1994)',
 'Beauty and the Beast (1991)',
 "Bug's Life, A (1998)",
 'Tarzan (1999)',
 'Toy Story 2 (1999)',
 'Hunchback of Notre Dame, The (1996)',
 'Little Mermaid, The (1989)',
 'Anastasia (1997)',
 'Nightmare Before Christmas, The (1993)',
 'James and the Giant Peach (1996)',
 'Iron Giant, The (1999)',
 'Pocahontas (1995)',
 'Prince of Egypt, The (1998)',
 'South Park: Bigger, Longer and Uncut (1999)',
 '101 Dalmatians (1961)',
 'Rescuers Down Under, The (1990)',
 'Babe (1995)',
 'Aladdin and the King of Thieves (1996)',
 'Cinderella (1950)',
 "Charlotte's Web (1973)"]

추천된 영화 모두 `Animation`이나 `Children's` 장르의 영화로 성향을 잘 파악하고 추천해 주는 것을 확인 할 수 있다.

__모델들은 왜 Lion King을 가장 추천해 주었을까?__<br>
AlternatingLeastSquares 클래스에 구현된 explain 메소드를 사용하면 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있다.

In [63]:
lionking = movie_to_idx['Lion King, The (1994)']
explain_lionking = als_model.explain(user, csr_data, itemid=lionking)

# 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됩니다.)
[(idx_to_movie[i[0]], i[1]) for i in explain_lionking[1]]

[('Aladdin (1992)', 0.2420083656883179),
 ('Antz (1998)', 0.10499965916134685),
 ('Mulan (1998)', 0.10389637616608918),
 ('Hercules (1997)', 0.07654569307218391),
 ('Toy Story (1995)', 0.05825559764228709)]

Aladdin에 대한 기여도가 가장 크다. Lion King도 뮤지컬 애니메이션이어서 일까?

https://danthetech.netlify.app/DataScience/evaluation-metrics-for-recommendation-system